In [1]:
import pandas as pd 

df = pd.read_csv('data1.csv')
df

,label,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,Whole,253,253,253,253,253,253,253,253,253,...,254,254,254,254,254,254,254,254,254,254
1,Whole,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,Whole,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,Whole,253,253,253,253,253,253,253,253,253,...,253,253,253,253,253,253,253,253,253,253
4,Whole,252,252,252,252,252,252,252,252,252,...,252,252,252,252,252,252,252,252,252,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Sixteenth,254,254,254,254,254,254,254,254,254,...,254,254,254,254,254,254,254,254,254,254
4996,Sixteenth,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4997,Sixteenth,253,253,253,253,253,253,253,253,253,...,253,253,253,253,253,253,253,253,253,253
4998,Sixteenth,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [2]:
df.label.value_counts()

label
Whole        1000
Half         1000
Quarter      1000
Eight        1000
Sixteenth    1000
Name: count, dtype: int64

In [3]:
from tensorflow.keras.utils import to_categorical

labels = df.label.map({'Whole': 0,'Half': 1,'Quarter': 2,'Eight': 3,'Sixteenth': 4})
labels = to_categorical(labels,num_classes = 5)
labels

2025-05-12 09:50:09.514743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747032609.659392    1091 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747032609.697749    1091 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747032610.047255    1091 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747032610.047302    1091 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747032610.047306    1091 computation_placer.cc:177] computation placer alr

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [4]:
input = df.drop(['label'],axis=1).values
input = input.reshape(-1, 64, 64, 1)
input.shape

(5000, 64, 64, 1)

Нормализуем данные

In [5]:
input = input.astype('float32') / 255.0

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_temp,y_train,y_temp = train_test_split(input,labels,test_size=0.4,random_state=22)
y_train.sum(axis=0)

array([607., 606., 593., 590., 604.])

In [7]:
x_val,x_test, y_val,y_test=train_test_split(x_temp,y_temp,train_size=0.5,random_state=22)
y_test.sum(axis=0)

array([188., 202., 211., 210., 189.])

In [8]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Flatten, Dense

inputs = Input(shape=(64, 64, 1))
x = Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = Flatten()(x)
outputs = Dense(5, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

2025-05-12 09:50:20.687327: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',
              metrics=['accuracy','precision','recall'])
model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))
loss, acc, prec, rec = model.evaluate(x_test,y_test)
print(f'acceracy: {acc},\n precision: {prec},\n recall: {rec}')


Epoch 1/10


94/94 ━━━━━━━━━━━━━━━━━━━━ 20s 170ms/step - accuracy: 0.9808 - loss: 0.0538 - precision: 0.9808 - recall: 0.9808 - val_accuracy: 0.9910 - val_loss: 0.0419 - val_precision: 0.9920 - val_recall: 0.9890
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 153ms/step - accuracy: 0.9983 - loss: 0.0138 - precision: 0.9983 - recall: 0.9983 - val_accuracy: 0.9870 - val_loss: 0.0566 - val_precision: 0.9870 - val_recall: 0.9860
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 21s 155ms/step - accuracy: 0.9929 - loss: 0.0273 - precision: 0.9929 - recall: 0.9929 - val_accuracy: 0.9880 - val_loss: 0.0378 - val_precision: 0.9890 - val_recall: 0.9880
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 151ms/step - accuracy: 0.9972 - loss: 0.0112 - precision: 0.9972 - recall: 0.9972 - val_accuracy: 0.9840 - val_loss: 0.0507 - val_precision: 0.9840 - val_recall: 0.9830
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 15s 153ms/step - accuracy: 0.9952 - loss: 0.0124 - precision: 0.9957 - recall: 0.9952 - val_accuracy: 0.9920 - val_loss: 0.0338 -

Проведем тесты с другими функциями потерь и оптимизаторами 

In [ ]:
model2 = Model(inputs=inputs, outputs=outputs)
model2.compile(optimizer='adam',loss='categorical_crossentropy',
              metrics=['accuracy','precision','recall'])
model2.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))
loss, acc, prec, rec = model2.evaluate(x_test,y_test)
print(f'acceracy: {acc},\n precision: {prec},\n recall: {rec}')


Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 16s 135ms/step - accuracy: 0.9942 - loss: 0.0222 - precision: 0.9942 - recall: 0.9942 - val_accuracy: 0.9950 - val_loss: 0.0179 - val_precision: 0.9960 - val_recall: 0.9950
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 146ms/step - accuracy: 0.9973 - loss: 0.0092 - precision: 0.9973 - recall: 0.9973 - val_accuracy: 0.9950 - val_loss: 0.0112 - val_precision: 0.9950 - val_recall: 0.9950
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - accuracy: 1.0000 - loss: 8.7842e-04 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9950 - val_loss: 0.0189 - val_precision: 0.9950 - val_recall: 0.9950
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 111ms/step - accuracy: 1.0000 - loss: 3.5816e-04 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9950 - val_loss: 0.0219 - val_precision: 0.9950 - val_recall: 0.9950
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 140ms/step - accuracy: 1.0000 - loss: 9.2509e-05 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.99

In [ ]:
model3 = Model(inputs=inputs, outputs=outputs)
model3.compile(optimizer='sgd',loss='categorical_crossentropy',
              metrics=['accuracy','precision','recall'])
model3.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))
loss, acc, prec, rec = model3.evaluate(x_test,y_test)
print(f'acceracy: {acc},\n precision: {prec},\n recall: {rec}')


Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 16s 157ms/step - accuracy: 1.0000 - loss: 3.8520e-05 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9920 - val_loss: 0.0364 - val_precision: 0.9920 - val_recall: 0.9920
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - accuracy: 1.0000 - loss: 3.5557e-05 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9920 - val_loss: 0.0365 - val_precision: 0.9920 - val_recall: 0.9920
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 1.0000 - loss: 2.5765e-05 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9920 - val_loss: 0.0365 - val_precision: 0.9920 - val_recall: 0.9920
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 138ms/step - accuracy: 1.0000 - loss: 3.0663e-05 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9920 - val_loss: 0.0366 - val_precision: 0.9920 - val_recall: 0.9920
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 1.0000 - loss: 3.5956e-05 - precision: 1.0000 - recall: 1.0000 - val_accura

In [ ]:
model4 = Model(inputs=inputs, outputs=outputs)
model4.compile(optimizer='rmsprop',loss='categorical_crossentropy',
              metrics=['accuracy','precision','recall'])
model4.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))
loss, acc, prec, rec = model4.evaluate(x_test,y_test)
print(f'acceracy: {acc},\n precision: {prec},\n recall: {rec}')


Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 16s 138ms/step - accuracy: 0.9981 - loss: 0.0046 - precision: 0.9981 - recall: 0.9981 - val_accuracy: 0.9770 - val_loss: 0.0765 - val_precision: 0.9770 - val_recall: 0.9770
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 119ms/step - accuracy: 0.9887 - loss: 0.0583 - precision: 0.9887 - recall: 0.9887 - val_accuracy: 0.9950 - val_loss: 0.0269 - val_precision: 0.9950 - val_recall: 0.9950
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 117ms/step - accuracy: 0.9998 - loss: 0.0020 - precision: 0.9998 - recall: 0.9998 - val_accuracy: 0.9920 - val_loss: 0.0187 - val_precision: 0.9920 - val_recall: 0.9920
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.9970 - loss: 0.0086 - precision: 0.9970 - recall: 0.9970 - val_accuracy: 0.9940 - val_loss: 0.0250 - val_precision: 0.9940 - val_recall: 0.9940
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 138ms/step - accuracy: 1.0000 - loss: 6.4168e-05 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9940 - val